In [1]:
import os
import toml
import pandas as pd
from utils.config_helper import update_nested_toml, load_config

breast
lung
prostate
stomach
rectal

In [2]:
TYPE = "stomach"
CONFIG_PATH = f"../config/{TYPE}.toml"
config = load_config(CONFIG_PATH)

In [3]:
def inspect_nan(df, name):
    print(df[pd.isna(df[name])])

In [4]:
beta_file_number = config["init"]["hyper"]["beta_file_number"]
test_ratio = config["init"]["hyper"]["test_ratio"]
seed = config["init"]["hyper"]["splitting_seed"]
normal_number_0 = config["init"]["hyper"]["normal_number_0"]
if beta_file_number == 2:
    normal_number_1 = config["init"]["hyper"]["normal_number_1"]
data_source = config["init"]["hyper"]["data_source"]
is_columns_duplicated = config["init"]["hyper"]["is_columns_duplicated"]
if data_source == "GDC_stomach_GSE99553":  # god forgive me
    is_columns_duplicated_1 = config["init"]["hyper"]["is_columns_duplicated_1"]

In [5]:
trainOutPath = f"../{TYPE}/result/{data_source}/train{int(100-test_ratio*100)}"
testOutPath = f"../{TYPE}/result/{data_source}/test{int(test_ratio*100)}"

### Section. 0 Merge and Split Champ Data 
(if there are more than one normalized beta data)

#### Summary
- beta data is split into train and test
- all beta data will the following format
  - column 0 is the id of the sample
  - column 1 to n is the beta value of each CpG site
  - normal samples come first, then tumor samples

In [ ]:
df0 = pd.read_csv(f"../{TYPE}/champ_result/{data_source}/all_beta_normalized_0.csv")

In [ ]:
if beta_file_number == 2:
    df1 = pd.read_csv(f"../{TYPE}/champ_result/{data_source}/all_beta_normalized_1.csv")

In [ ]:
# DEBUG
df0
# END

In [ ]:
# DEBUG
df1
# END

In [ ]:
# potential feature loss
if beta_file_number == 2:
    feature_name_0 = df0.iloc[:, 0].tolist()
    feature_name_1 = df1.iloc[:, 0].tolist()

    feature_name = list(set(feature_name_0).intersection(feature_name_1))
    update_nested_toml(
        "preprocess.merge_and_split", "feature_size_0", len(feature_name_0)
    )
    update_nested_toml(
        "preprocess.merge_and_split", "feature_size_1", len(feature_name_1)
    )
    update_nested_toml(
        "preprocess.merge_and_split", "feature_size_intersection", len(feature_name)
    )
elif beta_file_number == 1:
    feature_name = df0.iloc[:, 0].tolist()
    update_nested_toml(
        "preprocess.merge_and_split", "feature_size_0", len(feature_name)
    )

In [ ]:
if beta_file_number == 2:
    df0_join = df0[df0.iloc[:, 0].isin(feature_name)]
    df1_join = df1[df1.iloc[:, 0].isin(feature_name)]

In [ ]:
if beta_file_number == 2:
    df0_join = df0_join.iloc[:, 1::is_columns_duplicated]
    if data_source == "GDC_stomach_GSE99553":  # god forgive me
        df1_join = df1_join.iloc[:, 1::is_columns_duplicated_1]
    df0_join.reset_index(drop=True, inplace=True)
    df1_join.reset_index(drop=True, inplace=True)
    df0_join_normal = df0_join.iloc[:, :normal_number_0]
    df0_join_tumor = df0_join.iloc[:, normal_number_0:]
    df1_join_normal = df1_join.iloc[:, :normal_number_1]
    df1_join_tumor = df1_join.iloc[:, normal_number_1:]
elif beta_file_number == 1:
    df0_join = df0.iloc[:, 1::is_columns_duplicated]

In [ ]:
if beta_file_number == 2:
    df_normal = pd.concat([df0_join_normal, df1_join_normal], axis=1)
    df_tumor = pd.concat([df0_join_tumor, df1_join_tumor], axis=1)

In [ ]:
# drop those samples with missing value
# note: could use padding or other methods to fill the missing value

if beta_file_number == 2:
    update_nested_toml(
        "preprocess.merge_and_split", "Before_dropna_dfn_shape", df_normal.shape
    )
    update_nested_toml(
        "preprocess.merge_and_split", "Before_dropna_dfc_shape", df_tumor.shape
    )
    df_normal.dropna(inplace=True, axis=1)
    df_tumor.dropna(inplace=True, axis=1)
    update_nested_toml(
        "preprocess.merge_and_split", "After_dropna_dfn_shape", df_normal.shape
    )
    update_nested_toml(
        "preprocess.merge_and_split", "After_dropna_dfc_shape", df_tumor.shape
    )
elif beta_file_number == 1:
    update_nested_toml(
        "preprocess.merge_and_split", "Before_dropna_df_shape", df0_join.shape
    )
    df0_join.dropna(inplace=True, axis=1)
    update_nested_toml(
        "preprocess.merge_and_split", "After_dropna_df_shape", df0_join.shape
    )

In [ ]:
if beta_file_number == 2:
    df = pd.concat([df_normal, df_tumor], axis=1)
    df.columns = range(df.shape[1])
elif beta_file_number == 1:
    df = df0_join
    df.columns = range(df.shape[1])

In [ ]:
# DEBUG
df
# END

In [ ]:
config = load_config(CONFIG_PATH)
if beta_file_number == 1:
    normal_count = config["init"]["hyper"]["normal_number_0"]
elif beta_file_number == 2:
    normal_count = config["init"]["hyper"]["normal_number_0"] + config["init"]["hyper"]["normal_number_1"]
X = df.T
y = [(0 if i < normal_count else 1) for i in range((df.shape[1]))]

In [ ]:
from collections import Counter
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=test_ratio, random_state=seed
)

train_class_distribution = Counter(y_train)
testing_class_distribution = Counter(y_test)

update_nested_toml("preprocess.merge_and_split", "training_set_samples", len(X_train))
update_nested_toml("preprocess.merge_and_split", "testing_set_samples", len(X_test))
update_nested_toml(
    "preprocess.merge_and_split",
    "train_class_distribution",
    [train_class_distribution[0], train_class_distribution[1]],
)
update_nested_toml(
    "preprocess.merge_and_split",
    "testing_class_distribution",
    [testing_class_distribution[0], testing_class_distribution[1]],
)

In [ ]:
X_train.columns = feature_name
X_train["label"] = y_train
X_train = X_train.sort_values(by=["label"])
train_df = X_train.T
train_df.columns = range(train_df.shape[1])

X_test.columns = feature_name
X_test["label"] = y_test
X_test = X_test.sort_values(by=["label"])
test_df = X_test.T
test_df.columns = range(test_df.shape[1])

train_df.insert(0, "Unnamed: 0", train_df.index)
train_df.reset_index(drop=True, inplace=True)
test_df.insert(0, "Unnamed: 0", test_df.index)
test_df.reset_index(drop=True, inplace=True)

In [ ]:
# DEBUG
train_df
# END

In [ ]:
# DEBUG
test_df
# END

In [ ]:
os.makedirs(f"{trainOutPath}", exist_ok=True)
os.makedirs(
    f"{testOutPath}",
    exist_ok=True,
)

train_df.to_csv(f"{trainOutPath}/all_beta_normalized_0.csv", index=False)
test_df.to_csv(f"{testOutPath}/all_beta_normalized_1.csv", index=False)

In [ ]:
zip_filename = "beta_files.zip"

In [ ]:
import zipfile

with zipfile.ZipFile(zip_filename, "w") as zipf:
    zipf.write(
        f"{trainOutPath}/all_beta_normalized_0.csv", arcname="all_beta_normalized_0.csv"
    )
    zipf.write(
        f"{testOutPath}/all_beta_normalized_1.csv", arcname="all_beta_normalized_1.csv"
    )

In [ ]:
from api import utils
service = utils.authenticate_drive()

In [ ]:
directory = utils.create_folder(service, TYPE)

In [ ]:
utils.run_upload_with_separate_thread(service, directory, zip_filename)

In [ ]:
# downlaod the zip from drive
utils.download_file(service, directory, zip_filename)

### Sec. 1 dbeta calculation
#### Summary
- calculate the difference of beta value between tumor and normal samples
- the output file will have the following format
  - column 0 is the id of the sample
  - column 1 is the gene name
  - column 2 is the difference of beta value between tumor and normal samples

### Implementation
1. split normal and tumor samples
2. remove outliers in normal and tumor samples
3. calculate the mean of normal sammples
4. tumor - avg(normal)
5. calculate the mean of tumor samples
6. merge with DMP file
7. exclude the genes not in single comorbidity list

In [11]:
import gdown

##### Lung
https://drive.google.com/file/d/1BjBach5iyFb0n1DIG6Xc0ru3jMUtmlfW/view?usp=sharing
##### Rectal
https://drive.google.com/file/d/11DZAwbtqVriSN8EhUNhQEEwVyntiycRa/view?usp=sharing
##### Stomach
https://drive.google.com/file/d/1QBklKEO61ZYqxo-gfjBsWg3mcREsTRc7/view?usp=sharing
##### Prostate
tbd
##### Breast
tbd

In [ ]:
url = input()

output = 'download.zip'

In [ ]:
gdown.download(url, output, quiet=False)

In [13]:
import zipfile
import shutil

with zipfile.ZipFile("download.zip", "r") as zip_ref:
    zip_ref.extractall("download")

shutil.move("download/all_beta_normalized_0.csv", f"all_beta_normalized_0.csv")
shutil.move("download/all_beta_normalized_1.csv", f"all_beta_normalized_1.csv")

os.remove("download.zip")
shutil.rmtree("download")

In [ ]:
train_df = pd.read_csv(f"{trainOutPath}/all_beta_normalized_0.csv")

In [ ]:
def IQR(df):
    Q1 = df.quantile(0.25)
    Q3 = df.quantile(0.75)
    IQR = Q3 - Q1
    upper_fence = Q3 + IQR * 1.5
    lower_fence = Q1 - IQR * 1.5
    return upper_fence, lower_fence


def no_outlier(df):
    upper_fence, lower_fence = IQR(df)
    ddf = df[(df > lower_fence) & (df < upper_fence)]
    return ddf

In [ ]:
config = load_config(CONFIG_PATH)
normal_count = config["preprocess"]["merge_and_split"]["train_class_distribution"][0]
all_beta_normalized_normal = train_df.iloc[:-1, 1 : normal_count + 1 :]


all_beta_normalized_tumor = train_df.iloc[:-1, normal_count + 1 : :]

In [ ]:
all_beta_normalized_normal = no_outlier(all_beta_normalized_normal)
all_beta_normalized_tumor = no_outlier(all_beta_normalized_tumor)

In [ ]:
train_normal_avg = all_beta_normalized_normal.mean(skipna=True, axis=1)

In [ ]:
all_beta_normalized_tumor = (all_beta_normalized_tumor).subtract(
    train_normal_avg, axis=0
)

In [ ]:
all_beta_normalized_tumor = no_outlier(all_beta_normalized_tumor)

In [ ]:
train_tumor_mean = all_beta_normalized_tumor.mean(skipna=True, axis=1)

In [ ]:
delta_beta = pd.merge(
    train_df.iloc[:-1, :1],
    pd.DataFrame(train_tumor_mean, columns=["dbeta"]),
    left_index=True,
    right_index=True,
)
update_nested_toml("preprocess.dbeta", "delta_beta_avg", delta_beta.shape[0])

In [ ]:
# print(delta_beta[pd.isna(delta_beta["dbeta"])])
# record the list of feature with dbeta being NaN
update_nested_toml(
    "preprocess.dbeta",
    "NaN_dbeta_feature",
    delta_beta.loc[pd.isna(delta_beta["dbeta"]), "Unnamed: 0"].tolist(),
)
delta_beta.dropna(inplace=True, axis=0)
update_nested_toml("preprocess.dbeta", "delta_beta_avg_remove_NaN", delta_beta.shape[0])

In [ ]:
dmp = pd.read_csv(f"../{cancer_type}/champ_result/{data_source}/DMP_result_0.csv")
dmp = dmp[["Unnamed: 0", "gene", "feature"]]
update_nested_toml("preprocess.dbeta", "dmp_before_dropna_shape_feature", dmp.shape[0])
dmp.dropna(inplace=True)
update_nested_toml("preprocess.dbeta", "dmp_after_dropna_shape_feature", dmp.shape[0])

In [ ]:
result = pd.merge(delta_beta, dmp, on="Unnamed: 0", how="left")
update_nested_toml(
    "preprocess.dbeta", "delta_beta_avg_remove_NaN_with_gene_name", result.shape[0]
)

In [ ]:
def find_max_dBeta_grouped(group):
    idx_max = group["dbeta"].abs().idxmax()
    return group.loc[idx_max]


dbeta = result.groupby("gene", as_index=False).apply(
    find_max_dBeta_grouped, include_groups=False
)

In [ ]:
dbeta.columns = ["gene", "ID", "dbeta", "feature"]
dbeta = dbeta[["ID", "gene", "dbeta", "feature"]]
# DEBUG
dbeta
# END

In [ ]:
# comorbidity = pd.read_csv(
#     "../external_result/matchgene174_single_3Y10__OR2.txt", sep="\t", header=None
# )
# dbeta = dbeta[
#     dbeta["gene"].isin(comorbidity[0])
# ]

# result_max_per_gene_single

In [ ]:
dbeta.to_csv(f"{trainOutPath}/dbeta.csv", index=False)

### Sec. 2 Filter genes by dbeta values
1. filter genes by dbeta values
3. filter genes by TSS position
4. plot distribution of dbeta values
5. plot PCA for normal and tumor


In [ ]:
# dbeta = pd.read_csv(f"{trainOutPath}/dbeta.csv")

#### Filtering TSS

In [ ]:
TSS = dbeta[dbeta["feature"].str.contains("TSS")]

In [ ]:
TSS.to_csv(f"{trainOutPath}/dbeta_TSS.csv", index=False)

#### Thresholding

In [ ]:
threshold = 0.5
TSS_threshold = TSS[abs(TSS["dbeta"]) > threshold]
while True:
    TSS_threshold = TSS[abs(TSS["dbeta"]) > threshold]
    count = TSS_threshold.shape[0]
    if (
        config["preprocess"]["filtering"]["hyper"]["avg_dbeta_lower_bound"]
        <= count
        <= config["preprocess"]["filtering"]["hyper"]["avg_dbeta_upper_bound"]
    ):
        break
    threshold -= 0.01
threshold = round(threshold, 2)
update_nested_toml("preprocess.filtering", "threshold", threshold)

In [ ]:
TSS_threshold.to_csv(f"{trainOutPath}/dbeta_TSS_{threshold}.csv", index=False)

#### Visualization

In [ ]:
# DEBUG
import seaborn as sns
import matplotlib.pyplot as plt

sns.kdeplot(TSS_threshold["dbeta"])
plt.xlabel("delta Beta value")
plt.title("Density plot of delta Beta value")
# END

In [ ]:
# train_df = pd.read_csv(f"{trainOutPath}/all_beta_normalized_0.csv")

In [ ]:
normal_count = (train_df.iloc[-1, 1:] == 0).sum()
df_gene = train_df.iloc[:-1, :]
df_gene = df_gene[df_gene[df_gene.columns[0]].isin(dbeta["ID"])]
X = df_gene.iloc[:, 1:].reset_index(drop=True).T
y = [0 if i < normal_count else 1 for i in range(X.shape[0])]
# DEBUG
print(f"X shape: {X.shape}")
print(f"y shape: {len(y)}")
# END

In [ ]:
import plotly.express as px
import pandas as pd
from sklearn.decomposition import PCA

pca = PCA(n_components=3)
X_pca = pca.fit_transform(X)

df = pd.DataFrame(
    {
        "Principal Component 1": X_pca[:, 0],
        "Principal Component 2": X_pca[:, 1],
        "Principal Component 3": X_pca[:, 2],
        "Class": y,
    }
)
print(df.shape)
fig = px.scatter_3d(
    df,
    x="Principal Component 1",
    y="Principal Component 2",
    z="Principal Component 3",
    color="Class",
    title="PCA of Dataset",
    color_continuous_scale="Viridis",
)

fig.update_layout(
    scene=dict(
        xaxis_title="Principal Component 1",
        yaxis_title="Principal Component 2",
        zaxis_title="Principal Component 3",
    )
)

# fig.show()

fig.write_html(f"{trainOutPath}/preprocess_filtering_pca.html")
# open in browser

### Sec. 3 Machine Learning
1. remove hypo-methylated genes
2. RFE
3. RFECV (tbd)

In [ ]:
config = load_config(CONFIG_PATH)
threshold = config["preprocess"]["filtering"]["threshold"]
TSS_threshold = pd.read_csv(f"{trainOutPath}/dbeta_TSS_{threshold}.csv")

In [ ]:
TSS_threshold_hyper = TSS_threshold[TSS_threshold["dbeta"] > 0]
# DEBUG
TSS_threshold_hyper
# END

In [ ]:
train_df = pd.read_csv(f"{trainOutPath}/all_beta_normalized_0.csv")
test_df = pd.read_csv(f"{testOutPath}/all_beta_normalized_1.csv")

In [ ]:
X_train = train_df[train_df["Unnamed: 0"].isin(TSS_threshold_hyper["ID"])]
X_test = test_df[test_df["Unnamed: 0"].isin(TSS_threshold_hyper["ID"])]

In [ ]:
X_train = X_train.iloc[:, 1:].T.values.tolist()
X_test = X_test.iloc[:, 1:].T.values.tolist()

In [ ]:
config = load_config(CONFIG_PATH)
normal_count_train = config["preprocess"]["merge_and_split"][
    "train_class_distribution"
][0]
normal_count_test = config["preprocess"]["merge_and_split"][
    "testing_class_distribution"
][0]
y_train = [0 if i < normal_count_train else 1 for i in range(len(X_train))]
y_test = [0 if i < normal_count_test else 1 for i in range(len(X_test))]

In [ ]:
import pandas as pd
from sklearn.model_selection import cross_val_score
from sklearn.feature_selection import RFE
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from sklearn.metrics import (
    accuracy_score,
    recall_score,
    precision_score,
    f1_score,
    matthews_corrcoef,
    roc_auc_score,
    auc,
)

In [ ]:
def get_model(name):
    if name == "SVM":
        return SVC(kernel="linear")
    elif name == "LR":
        return LogisticRegression()
    elif name == "DT":
        return DecisionTreeClassifier()
    elif name == "RF":
        return RandomForestClassifier()
    elif name == "XGB":
        return XGBClassifier()
    else:
        raise ValueError(f"Unknown model name: {name}")


models = [
    "SVM",
    "LR",
    "DT",
    "RF",
    "XGB",
]


def append_to_file(file_name, data):
    if not os.path.isfile(file_name):
        data.to_csv(file_name, index=False)
    else:
        data.to_csv(file_name, index=False, mode="a", header=False)


for train_model_name in models:
    train_model = get_model(train_model_name)
    for feature_count in range(20, 100):
        rfe = RFE(estimator=train_model, n_features_to_select=feature_count)
        X_train_rfe = rfe.fit_transform(X_train, y_train)

        selected_feature_names = (
            pd.DataFrame(TSS_threshold_hyper.iloc[rfe.support_, 0])
            .reset_index(drop=True)
            .T
        )

        label = f"{train_model_name}_{feature_count}"
        selected_feature_names.insert(0, "train_model_name", label)

        append_to_file(f"{trainOutPath}/selected_feature_names.csv", selected_feature_names)

        for test_model_name in models:
            test_model = get_model(test_model_name)
            test_model.fit(X_train_rfe, y_train)

            # cross_val_score: Evaluate a score by cross-validation with accuracy as the scoring method
            train_accuracy_cv = cross_val_score(
                test_model, X_train_rfe, y_train, cv=5, scoring="accuracy"
            ).mean()

            # performace on test set
            X_test_rfe = rfe.transform(X_test)
            y_pred = test_model.predict(X_test_rfe)
            X_test_df = pd.DataFrame(X_test)
            incorrect_indices = X_test_df.index[y_pred != y_test]

            accuracy = accuracy_score(y_test, y_pred)
            precision = precision_score(y_test, y_pred)
            recall = recall_score(y_test, y_pred)
            f1 = f1_score(y_test, y_pred)
            mcc = matthews_corrcoef(y_test, y_pred)
            # 1 is perfect prediction, -1 is imperfect prediction, 0 is equal to random prediction
            fpr, tpr, _ = roc_curve(y_test, y_pred)
            roc_auc = auc(fpr, tpr)

            new_performance_row = pd.DataFrame(
                [
                    {
                        "train_model": train_model_name,
                        "test_model": test_model_name,
                        "features": feature_count,
                        "AUC": roc_auc,
                        "accuracy (5-fold-Cross-Validation)": train_accuracy_cv,
                        "accuracy": accuracy,
                        "precision": precision,
                        "recall": recall,
                        "f1_score": f1,
                        "MCC": mcc,
                        "J-index": recall + accuracy - (1 - recall) - 1,
                        "incorrect predictions count": len(incorrect_indices),
                    }
                ]
            )
            append_to_file(f"{trainOutPath}/results.csv", new_performance_row)

            new_fpr_tpr_row = pd.DataFrame(
                [
                    {
                        "train_model": train_model_name,
                        "test_model": test_model_name,
                        "features": feature_count,
                        "fpr": fpr,
                        "tpr": tpr,
                        "AUC": roc_auc,
                    }
                ]
            )
            append_to_file(f"{trainOutPath}/fpr_tpr.csv", new_fpr_tpr_row)